#### Sociology 128D: Mining Culture Through Text Data: Introduction to Social Data Science

# Notebook 10: Topic Modeling with Gensim

In this notebook, we will take our first look at topic modeling, which we will continue to explore over the next few class sessions. Topic modeling has received considerable attention within the social sciences and humanities because it can help us to identify latent themes ("topics") in a corpus.

Rather than assigning documents to discrete categories and saying they are about only one thing, topic modeling allows us to inductively identify latent topics and see that documents are about different topics to different extents. The approach we will focus on is called [latent Dirichlet allocation (LDA)](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation). The key idea is that documents are mixtures of topics, and topics are mixtures of words.

The figure below, taken from [Blei 2012](https://doi.org/10.1145/2133806.2133826), illustrates the basic intuition of LDA. There are two further points to emphasize: The researcher must choose the number of topics, and each word is associated with each topic to some extent. Words, like documents, may be primarily associated with one topic, or they may instead be associated with multiple topics to a substantial degree. Topics are often (helpfully) thought of as groups of words but, in reality, they are more like groups of *weights* on words. (In this context, "weight" means a number representing the strength of the association between the word and the topic.)

<!-- [![blei_2012_fig1](https://raw.githubusercontent.com/soc128d/soc128d.github.io/master/assets/images/blei_2012_fig1.png)](https://doi.org/10.1145/2133806.2133826) -->

<a href="https://doi.org/10.1145/2133806.2133826"><img src="https://raw.githubusercontent.com/soc128d/soc128d.github.io/master/assets/images/blei_2012_fig1.png" width=800 height=508></a>
<div style="text-align: right"><a href="https://doi.org/10.1145/2133806.2133826">Source: Blei 2012</a></div>

## Setup

You will need to install `gensim` if you have not. You will also need to install `pyLDAvis`. Importing the latter may raise a number of warnings. For purposes of this notebook, you can disregard them. The cell below uses the `warnings` library to filter various warnings, but this is added at the end because importing certain libraries can reset the filter settings. Importing `pyLDAvis` appears to do this. (Side note: This is a good time for a reminder that python code is "interpreted" rather than compiled--everything happens one line at a time, in order.) To make the notebook tidier, you can rerun the cell below to get rid of most of the warnings.

In [ ]:
import numpy as np
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models
import os
import warnings

from gensim.corpora.dictionary import Dictionary
from gensim.models.callbacks import CoherenceMetric, PerplexityMetric
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.ldamodel import LdaModel
from gensim.models.phrases import Phrases

pyLDAvis.enable_notebook()

warnings.filterwarnings("ignore", category=DeprecationWarning)

## Data

The dataset for this notebook is a (fixed!) preprocessed version of the Reddit r/jobs corpus we have used before. You can find the file on Canvas (<tt>Files -> Data -> rjobs_preprocessed_v2.json</tt>).

This corpus comprises posts submitted to [r/jobs](https://www.reddit.com/r/jobs/) between the morning of January 1, 2020, and the morning of January 1, 2021. The <tt>text</tt> field is the original title combined with the original text of the body of the post (that is, the <tt>title</tt> and <tt>selftext</tt> fields in other files we have used). The <tt>preprocessed</tt> field is a version of the text that has had URLs removed, contractions expanded (e.g., "shan't" -> "shall not"), stop words and single characters removed, and ordinal numbers converted to text (e.g., "1st" -> "first"). Remaining tokens have been lemmatized.

In [ ]:
df = pd.read_json("rjobs_preprocessed_v2.json")

In [ ]:
df.head()

In [ ]:
df.shape

## Identifying *n*-grams

The preprocessing we have already done is intended to convert variants of words (e.g., "ran" or "running") to a common version (e.g., "run") and remove words that will not help us learn about the meaning of documents. To this point, however, we have primarily focused on *unigrams*, or single-word tokens. Now we are going to identify *n*-grams. Often, we are interested in ideas with names longer than a single token, for example "work-life balance." Tools like topic modeling trained only on unigrams have a chance to identify topics strongly associated with the words *work*, *life*, and *balance*, but--just as with our other preprocessing steps--we can guide the algorithm to "read" in a particular way.

To identify *n*-grams, we are going to use [`gensim`'s `Phrases()` method](https://radimrehurek.com/gensim/models/phrases.html#gensim.models.phrases.Phrases). This allows us to identify bigrams, or frequent two-word sequences. If we `Phrases()` once, it may identify a bigram like "work_life." If we use it a second time, it may detect that "work_life" and "balance" occur next to one another frequently, and identify the trigram "work_life_balance" as a result.

To get started, we split the preprocessed text on whitespace by applying `str.split()` to the <tt>preprocessed</tt> column of our dataframe. We will add the `.tolist()` method to convert the column to a list, which we will assign to the variable <tt>preprocessed</tt>. The result is a list. Each element corresponds to one of the original posts, but it is now a list of tokens.

In [ ]:
preprocessed = df.preprocessed.apply(str.split).tolist()

In [ ]:
preprocessed[0]

Let's take a look at the function below. <tt>train_ngram_model()</tt> is a helper function that will simplify using `Phrases()` to identify *n*-grams in our corpus.

The line

```python
docs = [doc for doc in docs if doc]
```

is a list comprehension that simply returns every document ("doc") in the list we provide (<tt>docs</tt>) if the document exists. This gets rid of a few documents that, after preprocessing, do not have any words.

The next two lines (1) identify bigrams based on the documents we provide and (2) apply that knowledge to the documents by linking the words of a bigram with an underscore.

```python
bigram_model = Phrases(docs, min_count=min_count)
ngrams = bigram_model[docs]
```

We then cast the result as a list, using the variable name, <tt>ngrams</tt>.

```python
ngrams = list(ngrams)
```

If `inc_trigrams` is `True`, the process is repeated, this time using the documents that now may have bigrams already. Applying `Phrases()` a second time can identify trigrams--or even four-grams.

In [ ]:
def train_ngram_model(docs: list, min_count: int=5, inc_trigrams: bool=True) -> list:
    """Returns documents with n-grams joined by underscores"""
    docs = [doc for doc in docs if doc] # the "if doc" condition removes empty strings (docs with no words)
    bigram_model = Phrases(docs, min_count=min_count)
    ngrams = bigram_model[docs]
    ngrams = list(ngrams)
    if inc_trigrams:
        trigram_model = Phrases(ngrams, min_count=min_count)
        ngrams = trigram_model[ngrams]
        ngrams = list(ngrams)
    return ngrams

In [ ]:
ngrams = train_ngram_model(preprocessed, min_count=25)

Let's take a look at a hand-picked example. You can modify the function to change the minimum number of times a bigram needs to occur, and you can also look at the [documentation](https://radimrehurek.com/gensim/models/phrases.html#gensim.models.phrases.Phrases) for other arguments you can change to affect the behavior and results. Assuming you haven't modified things too much, the example below should include "work_life_balance."

In [ ]:
print(ngrams[753])

Now let's take a quick look at the size of the vocabulary, the number of *n*-grams in the vocabulary, and some random examples of bigrams and trigrams.

In [ ]:
vocab = set([word for doc in ngrams for word in doc])
print(len(vocab))
vocab_ngrams = list(filter(lambda x: "_" in x, vocab))
print(len(vocab_ngrams))
print(np.random.choice(vocab_ngrams, 25))

## LDA as a Bag-of-Words Model

LDA is sometimes called a bag-of-words model because it ignores word order. Each document may as well be a bag of words, the order of which could get shuffled around. This is because LDA looks at *word counts within documents*. Like other methods we've discussed, LDA is based on patterns of word co-occurrence, but "co-occurrence" in this case is defined as co-occurring in a document. (Contrast this, for example, to word embedding models based on "windows" of context words to the left or right of a target word.)

We effectively treated documents as bags of words when we used tf-idf weighting to compare document vectors as well.

[`Dictionary()`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary) does the work that a document-term matrix would do: It constructs a mapping of types (unique words) to indices and provides a method (`doc2bow()`) that converts documents to word counts.

In [ ]:
dictionary = Dictionary(ngrams)
print(dictionary[0])
print(len(dictionary))

Before we use `doc2bow()`, we'll filter out rare or exceedingly frequent words based on their document frequencies. [`Dictionary.filter_extremes()`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.filter_extremes) can be used to get rid of rare or frequent words, to control the overall size of the vocabulary, and to make sure you don't remove words you are interested in. It takes four optional arguments:
- no_below: An integer, the minimum document frequency
- no_above: A float between 0.0 and 1.0, the maximum proportion of documents any word should appear in
- keep_n: An integer, the number of types (unique words) to keep, starting with the most frequent
- keep_tokens: An iterable (like a list) or a string containing words to keep in the vocabulary, even if they are too rare or frequent according to the values you provide for the other arguments

In [ ]:
dictionary.filter_extremes(no_below=5, no_above=0.99, keep_tokens=["work_life_balance",
                                                                   "social_security_number"])
print(len(dictionary))

We can also save the dictionary, as with many `gensim` objects.

In [ ]:
dictionary.save("rjobs_gensim_dict.d")

Let's take a look at a sample document. We'll compare the original version to the bag-of-words version.

In [ ]:
sample_doc = df.loc[10619]

In [ ]:
print(f"ORIGINAL:\n{sample_doc.text}\n")
print(f"BAG OF WORDS:\n{dictionary.doc2bow(sample_doc.preprocessed.split())}")

We can look up one of these indices in <tt>dictionary</tt> to see the corresponding word:

In [ ]:
dictionary[7985]

Now we'll create a new version of our corpus, which we will just call <tt>corpus</tt> in line with the `gensim` documentation. We'll use the `.doc2bow()` method inside a list comprehension to convert each of our documents in the list <tt>ngrams</tt> to this bag of words format.

In [ ]:
corpus = [dictionary.doc2bow(doc) for doc in ngrams]

In [ ]:
len(corpus)

## LDA Implementations in Gensim

`gensim` has two implementations of LDA: [`LdaModel()`](https://radimrehurek.com/gensim/models/ldamodel.html#gensim.models.ldamodel.LdaModel) and [`LdaMulticore()`](https://radimrehurek.com/gensim/models/ldamulticore.html#gensim.models.ldamulticore.LdaMulticore). The latter is designed with parallelization in mind. With a large corpus, a complicated model, or a high number of iterations, `LdaMulticore()` may be the faster option. The two implementations generally do the same thing. We will talk about one difference next time (spoiler: the argument for the hyperparameter <tt>alpha</tt> cannot be set to "auto" when using `LdaMulticore()`).

The number of topics (*k*) is the one hyperparameter we really cannot ignore (not that we should ignore hyperparameters as a rule). Although LDA is an unsupervised machine learning method, choosing the number of topics really highlights the role of human judgment. LDA won't decide on the number of latent topics for us; but once we tell it to find *k* topics, it will do its best to identify latent themes. 

In [ ]:
k = 10 # number of topics

%time ldamulti = LdaMulticore(corpus, id2word=dictionary, num_topics=k, iterations=10, workers=os.cpu_count()-2)

%time lda = LdaModel(corpus, id2word=dictionary, num_topics=k, iterations=10)

Note: LDA is often associated with [Gibbs sampling](https://en.wikipedia.org/wiki/Gibbs_sampling), but the implementations in `gensim` use [variational Bayes](https://en.wikipedia.org/wiki/Variational_Bayesian_methods) ([see here](https://www.di.ens.fr/~fbach/mdhnips2010.pdf)). For a concise discussion of differences on Stack Exchange with a link to a review paper, [see here](https://stats.stackexchange.com/a/271862). The TL;DR is that we may want to use Gibbs sampling if compute time is not an issue, but variational Bayes can be faster.

Next time we will discuss various arguments that `LdaModel()` and `LdaMulticore()` accept as we look at hyperparameter tuning. For now, let's just take a look at some results. Often we assess topics by looking at the words that are most strongly associated with them. Here we will take a look at the `.print_topics()` method before turning to the more appealing `pyLDAvis`, which allows us to interactively explore the resulting topics.

In [ ]:
lda.print_topics()

In [ ]:
pyLDAvis.gensim_models.prepare(topic_model=lda, corpus=corpus, dictionary=dictionary, mds="tsne")